In [3]:
import numpy as np
import pandas as pd
import os
import sys
from collections import defaultdict
from importlib import reload
from bs4 import BeautifulSoup
import requests

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Getting Track-Artist-Album-Producer Data from a Wikipedia "List of Songs Produced by PERSON-X Page"

## Step 1: Retrieving a list of URLs from a Wikipedia list page

We want to scrape all of the URLs for the Wikipedia pages on songs produced by George Martin from this aggregate page: https://en.wikipedia.org/wiki/Category:Song_recordings_produced_by_George_Martin. We locate the relevant links and return a list of URL strings.

In [8]:
requests.get('https://en.wikipedia.org/wiki/Category:Song_recordings_produced_by_George_Martin').content.decode()[:1000]

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Category:Song recordings produced by George Martin - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Category:Song_recordings_produced_by_George_Martin","wgTitle":"Song recordings produced by George Martin","wgCurRevisionId":577742163,"wgRevisionId":577742163,"wgArticleId":38853525,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["George Martin","Song recordings by producer"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaul

In [10]:
html = requests.get('https://en.wikipedia.org/wiki/Category:Song_recordings_produced_by_George_Martin').content
soup = BeautifulSoup(html, 'html.parser')

In [27]:
#select the url for one link
soup.select('li')[3].select('a')[0]['href']

'/wiki/Act_Naturally'

In [28]:
#The full URL is https://en.wikipedia.org/wiki/Act_Naturally

url_test = 'https://en.wikipedia.org' + soup.select('li')[3].select('a')[0]['href']
url_test

'https://en.wikipedia.org/wiki/Act_Naturally'

The full list is in a `<div>` section where `class = "mw-category"`

In [33]:
martin_urls = []
domain = 'https://en.wikipedia.org'
links = soup.find_all('div', class_="mw-category")[0].find_all('a')
for link in links:
    path = link['href']
    url = domain + path
    martin_urls.append(url)
    
    
#NEED TO ADD FUNCTIONALITY FOR MULTIPLE PAGES

In [35]:
martin_urls[:6]

['https://en.wikipedia.org/wiki/12-Bar_Original',
 'https://en.wikipedia.org/wiki/Across_the_Universe',
 'https://en.wikipedia.org/wiki/Act_Naturally',
 'https://en.wikipedia.org/wiki/Alfie_(Burt_Bacharach_song)',
 'https://en.wikipedia.org/wiki/All_I%27ve_Got_to_Do',
 'https://en.wikipedia.org/wiki/All_My_Loving']

Success! Now on to...

## Step 2: Retrieving Track - Artist - Album - Producer from a Song's Wikipedia Page